In [5]:
import pandas as pd
import asyncio
import nbformat
from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor
from ipyleaflet import Map, Marker, Popup, AwesomeIcon, WidgetControl, LayersControl
import ipywidgets as widgets
from pyproj import Transformer
from IPython.display import display
from IPython.display import HTML as HTML_display
from ipywidgets import HTML, Checkbox, VBox, HBox, Layout

# Setze das Event-Loop-Policy für Windows
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

# Definiere den Transformer für UTM Zone 32N zu WGS84
transformer = Transformer.from_crs("EPSG:32632", "EPSG:4326")

# Initialisiere die Karte
m = Map(center=(51.961668, 7.624951), zoom=12, layout=dict(width='100%', height='600px'))

# Erstelle ein HTML-Widget für die Überschrift
title = widgets.HTML('<h2>Kinder- und Jugendbildungseinrichtungen Münster</h2>')

# Verwende ein VBox-Layout, um die Überschrift und die Karte anzuzeigen
display(widgets.VBox([title, m]))

# Erstelle Listen für die verschiedenen Marker-Typen
bibliothek_markers = []
buecherbus_markers = []
kita_markers = []
Förderschule_markers = []
Grundschule_markers = []
Primusschule_markers = []
Hauptschule_markers = []
Realschule_markers = []
Gesamtschule_markers = []
Gymnasium_markers = []
#-------------------------------------------------------------------------------------------------------------------------#
# Büchereidaten einlesen und organisieren
DataBiblio = []
count = 0
with open("Buechereien.csv", "r") as textFile:
    TextLines = textFile.readlines()
    TextLines = TextLines[1:]  # Entferne die erste Zeile
    for Zeile in TextLines:
        count += 1
        Zeile = Zeile.split("\t")  # Excel trennt Zeilen bei CSV-Export mit Tabs
        DataBiblio.append(Zeile)

Bibliodaten = {}
Biblio = {}
for Zeile in DataBiblio:
    Biblio = {}
    Biblio["Büchereiname"] = Zeile[0]
    Biblio["Adresse"] = Zeile[1:3]
    Biblio["Stadtteil"] = Zeile[3]
    Biblio["Mailadresse"] = Zeile[4]
    Biblio["Öffnungszeiten"] = Zeile[5]
    Biblio["Telefonnummer"] = Zeile[6]
    Biblio["Angebot"] = Zeile[7]
    Biblio["Zusatzservice"] = Zeile[8]
    Biblio["Rechtswert"] = Zeile[9]
    Biblio["Hochwert"] = Zeile[10]
    Biblio["Homepage"] = Zeile[11]
    Biblio["Medien"] = Zeile[12]
    Biblio["Fläche"] = Zeile[13]
    
    # Umrechnung von UTM-32 zu WGS84
    if Biblio["Rechtswert"] and Biblio["Hochwert"]:
        rechtswert = float(Biblio["Rechtswert"])
        hochwert = float(Biblio["Hochwert"])
        lat, lon = transformer.transform(rechtswert, hochwert)  # Reihenfolge angepasst
        Biblio["WGS84_Latitude"] = lat
        Biblio["WGS84_Longitude"] = lon
    
    #print(Biblio.get("WGS84_Latitude"))
    #print(Biblio.get("WGS84_Longitude"))

# Füge Marker an den Positionen der Büchereien in Münster hinzu
for zeile in DataBiblio:
    if zeile[9] and zeile[10]:  # Überprüfe, ob die Koordinaten nicht leer sind
        try:
            rechtswert = float(zeile[9])
            hochwert = float(zeile[10])
            lat, lon = transformer.transform(rechtswert, hochwert)  # Reihenfolge angepasst
#            print(f"Adding marker at: Latitude {lat}, Longitude {lon}")  # Debugging: Koordinaten ausgeben
            adresse = ", ".join(zeile[1:3])  # Adresse in einen String umwandeln
            popup_text = (f"<b>Bücherei:</b> {zeile[0]}<br>"
                          f"<b>Adresse:</b> {adresse}<br>"
                          f"<b>Öffnungszeiten:</b> {zeile[5]}<br>"
                          f"<b>Homepage:</b> <a href='{zeile[11]}' target='_blank'>{zeile[11]}</a>")
            popup = Popup(child=widgets.HTML(value=popup_text), location=(lat, lon))
            icon = AwesomeIcon(name='book', marker_color='darkblue', icon_color='white')  # Icon Bücherei
            markerBüchereien = Marker(location=(lat, lon), draggable=False, popup=popup, icon=icon)
            bibliothek_markers.append(markerBüchereien)
            m.add_layer(markerBüchereien)

        except Exception as e:
            print(f"Error adding marker: {e}")
#-------------------------------------------------------------------------------------------------------------------------#
# Bücherbusdaten einlesen und organisieren
DataBabobus = []
count=0
with open("Buecherbus.csv","r") as textFile:
    TextLines = textFile.readlines()
    TextLines = TextLines[1:]  # Entferne die erste Zeile
    for Zeile in TextLines:
        count += 1
        Zeile = Zeile.split("\t") # Excel trennt Zeilen bei CSV-Export mit Tabs
        #print(count, Zeile)
        DataBabobus.append(Zeile)   
#print(DataBabobus)

Babobusdaten = {}
Babobussi = {}
for Zeile in DataBabobus:
    Babobussi = {}
    Babobussi["Stadtteil"] = Zeile[0]
    Babobussi["Haltestelle"] = Zeile[1]
    Babobussi["Rechtswert"] = Zeile[2]
    Babobussi["Hochwert"] = Zeile[3]
    Babobussi["Homepage"] = Zeile[4]
    Babobussi["Fahrplan"] = Zeile[5]

    # Umrechnung von UTM-32 zu WGS84
    if Babobussi["Rechtswert"] and Babobussi["Hochwert"]:
        rechtswert = float(Babobussi["Rechtswert"])
        hochwert = float(Babobussi["Hochwert"])
        lat, lon = transformer.transform(rechtswert, hochwert)  # Reihenfolge angepasst
        Babobussi["WGS84_Latitude"] = lat
        Babobussi["WGS84_Longitude"] = lon
    
    #print(Babobussi.get("WGS84_Latitude"))
    #print(Babobussi.get("WGS84_Longitude"))

# Füge Marker an den Positionen der Bücherbushaltestellen in Münster hinzu
for zeile in DataBabobus:
    if zeile[2] and zeile[3]:  # Überprüfe, ob die Koordinaten nicht leer sind
        try:
            rechtswert = float(zeile[2])
            hochwert = float(zeile[3])
            lat, lon = transformer.transform(rechtswert, hochwert)  # Reihenfolge angepasst
#            print(f"Adding marker at: Latitude {lat}, Longitude {lon}")  # Debugging: Koordinaten ausgeben
            haltestelle = ", ".join(zeile[0:2])  # Adresse in einen String umwandeln
            popup_text = (f"<b>Haltestelle:</b> {haltestelle}<br>"            
                          f"<b>Fahrplan:</b> <a href='{zeile[5]}' target='_blank'>{zeile[5]}</a>"
                          f"<b>Homepage:</b> <a href='{zeile[4]}' target='_blank'>{zeile[4]}</a>")
            popup = Popup(child=widgets.HTML(value=popup_text), location=(lat, lon))
            icon = AwesomeIcon(name='bus', marker_color='green', icon_color='white')  # Icon für eine Haltestelle
            markerBücherbus = Marker(location=(lat, lon), draggable=False, popup=popup, icon=icon)
            buecherbus_markers.append(markerBücherbus)
            m.add_layer(markerBücherbus)

        except Exception as e:
            print(f"Error adding marker: {e}")
#-------------------------------------------------------------------------------------------------------------------------#
# Kitadaten einlesen und organisieren
DataKitas = []
count=0
with open("kitas_ms.csv","r") as textFile:
    TextLines = textFile.readlines()
    TextLines = TextLines[1:]  # Entferne die erste Zeile
    for Zeile in TextLines:
        count += 1
        Zeile = Zeile.split("\t") # Excel trennt Zeilen bei CSV-Export mit Tabs
        DataKitas.append(Zeile)  

Kitadaten = {}
Kita = {}
for Zeile in DataKitas:
    Kita = {}
    Kita["Kitaname"] = Zeile[0]
    Kita["Adresse"] = Zeile[1:3]
    Kita["Stadtteil"] = Zeile[3]
    Kita["Rechtswert"] = Zeile[4]
    Kita["Hochwert"] = Zeile[5]
    Kita["Homepage"] = Zeile[6]

    # Umrechnung von UTM-32 zu WGS84
    if Kita["Rechtswert"] and Kita["Hochwert"]:
        rechtswert = float(Kita["Rechtswert"])
        hochwert = float(Kita["Hochwert"])
        lat, lon = transformer.transform(rechtswert, hochwert)  # Reihenfolge angepasst
        Kita["WGS84_Latitude"] = lat
        Kita["WGS84_Longitude"] = lon
    
    #print(Kita.get("WGS84_Latitude"))
    #print(Kita.get("WGS84_Longitude"))

# Füge Marker an den Positionen der Kitas in Münster hinzu
for zeile in DataKitas:
    if zeile[4] and zeile[5]:  # Überprüfe, ob die Koordinaten nicht leer sind
        try:
            rechtswert = float(zeile[4])
            hochwert = float(zeile[5])
            lat, lon = transformer.transform(rechtswert, hochwert)  # Reihenfolge angepasst
#            print(f"Adding marker at: Latitude {lat}, Longitude {lon}")  # Debugging: Koordinaten ausgeben
            adresse = ", ".join(zeile[1:3])  # Adresse in einen String umwandeln
            popup_text = (f"<b>Kita:</b> {zeile[0]}<br>"
                          f"<b>Adresse:</b> {adresse}<br>")
            if zeile[6]:
                popup_text += f"<b>Homepage:</b> <a href='{zeile[6]}' target='_blank'>{zeile[6]}</a>"
            popup = Popup(child=widgets.HTML(value=popup_text), location=(lat, lon))
            icon = AwesomeIcon(name='rainbow', marker_color='purple', icon_color='white')  # Icon für eine Kita
            markerKita = Marker(location=(lat, lon), draggable=False, popup=popup, icon=icon)
            kita_markers.append(markerKita)
            m.add_layer(markerKita)

        except Exception as e:
            print(f"Error adding marker: {e}")
            
#-------------------------------------------------------------------------------------------------------------------------#
# Förderschuldaten einlesen und organisieren
DataFörderschulen = []
count=0
with open("Foerderschule_DataFrame.csv","r") as textFile:
    TextLines = textFile.readlines()
    TextLines = TextLines[1:]  # Entferne die erste Zeile
    for Zeile in TextLines:
        count += 1
        Zeile = Zeile.split("\t") # Excel trennt Zeilen bei CSV-Export mit Tabs
        DataFörderschulen.append(Zeile)  

Förderschuldaten = {}
Förderschule = {}
for Zeile in DataFörderschulen:
    Förderschule = {}
    Förderschule["Förderschulname"] = Zeile[1]
    Förderschule["Adresse"] = Zeile[2:4]
    Förderschule["Stadtteil"] = Zeile[5]
    Förderschule["Rechtswert"] = Zeile[6]
    Förderschule["Hochwert"] = Zeile[7]

# Füge Marker an den Positionen der Förderschulen in Münster hinzu
for zeile in DataFörderschulen:
    if len(zeile) > 7:  # Überprüfe, ob die Liste mindestens 5 Elemente enthält
        if zeile[6] and zeile[7]:  # Überprüfe, ob die Koordinaten nicht leer sind
            try:
                rechtswert = float(zeile[6])
                hochwert = float(zeile[7])
                #print(f"Adding marker at: Latitude {lat}, Longitude {lon}")  # Debugging: Koordinaten ausgeben
                adresse = ", ".join(zeile[2:4])  # Adresse in einen String umwandeln
                popup_text = (f"<b>Förderschule:</b> {zeile[0]}<br>"
                              f"<b>Adresse:</b> {adresse}<br>")
                popup = Popup(child=widgets.HTML(value=popup_text), location=(rechtswert, hochwert))
                icon = AwesomeIcon(name='school', marker_color='orange', icon_color='white')  # Icon für eine Schule
                markerFörderschule = Marker(location=(rechtswert, hochwert), draggable=False, popup=popup, icon=icon)
                Förderschule_markers.append(markerFörderschule)
                m.add_layer(markerFörderschule)
                # Weitere Verarbeitung
            except ValueError:
                # Fehlerbehandlung für ungültige Koordinaten
                print(f"Ungültige Koordinaten in Zeile: {zeile}")
                pass
    else:
        print(f"Unvollständige Zeile: {zeile}")

#-------------------------------------------------------------------------------------------------------------------------#
# Grundschulddaten einlesen und organisieren
DataGrundschulen = []
count = 0
with open("Grundschule_DataFrame_Lena.csv", "r") as textFile:
    TextLines = textFile.readlines()
    TextLines = TextLines[1:]  # Entferne die erste Zeile
    for Zeile in TextLines:
        count += 1
        Zeile = Zeile.split("\t")  # Excel trennt Zeilen bei CSV-Export mit Tabs
        DataGrundschulen.append(Zeile)

Grundschuldaten = {}
Grundschule = {}
for Zeile in DataGrundschulen:
    Grundschule = {}
    Grundschule["Grundschulname"] = Zeile[1]
    Grundschule["Adresse"] = Zeile[3]
    Grundschule["Stadtteil"] = Zeile[4]
    Grundschule["Rechtswert"] = Zeile[5]
    Grundschule["Hochwert"] = Zeile[6]

# Füge Marker an den Positionen der Grundschulen in Münster hinzu
Grundschule_markers = []
for zeile in DataGrundschulen:
    if len(zeile) > 6:  # Überprüfe, ob die Liste mindestens 5 Elemente enthält
        if zeile[5] and zeile[6]:  # Überprüfe, ob die Koordinaten nicht leer sind
            try:
                rechtswert = float(zeile[5])
                hochwert = float(zeile[6])
                adresse = ", ".join(zeile[3:5])  # Adresse in einen String umwandeln
                popup_text = (f"<b>Grundschule:</b> {zeile[0]}<br>"
                              f"<b>Adresse:</b> {adresse}<br>")
                popup = Popup(child=widgets.HTML(value=popup_text), location=(rechtswert, hochwert))
                icon = AwesomeIcon(name='school', marker_color='black', icon_color='white')  # Icon für eine Schule
                markerGrundschule = Marker(location=(rechtswert, hochwert), draggable=False, popup=popup, icon=icon)
                Grundschule_markers.append(markerGrundschule)
                m.add_layer(markerGrundschule)
            except ValueError:
                print(f"Ungültige Koordinaten in Zeile: {zeile}")
                pass
    else:
        print(f"Unvollständige Zeile: {zeile}")

#-------------------------------------------------------------------------------------------------------------------------#
# Primusschuldaten einlesen und organisieren
DataPrimusschulen = []
with open("Primusschule_DataFrame_Lena.csv", "r") as textFile:
    TextLines = textFile.readlines()
    TextLines = TextLines[1:]  # Entferne die erste Zeile
    for Zeile in TextLines:
        Zeile = Zeile.strip().split("\t")  # Entferne Leerzeichen und trenne Zeilen bei Tabs
        DataPrimusschulen.append(Zeile)

Primusschuldaten = {}
Primusschule = {}
for Zeile in DataPrimusschulen:
    if len(Zeile) > 8:  # Überprüfe, ob die Liste mindestens 9 Elemente enthält
        Primusschule = {}
        Primusschule["Primusschulname"] = Zeile[1]
        Primusschule["Adresse"] = Zeile[2:4]
        Primusschule["Stadtteil"] = Zeile[5]
        Primusschule["Rechtswert"] = Zeile[6]
        Primusschule["Hochwert"] = Zeile[7]

        # Füge Marker an den Positionen der Primusschulen in Münster hinzu
        try:
            rechtswert = float(Primusschule["Rechtswert"])
            hochwert = float(Primusschule["Hochwert"])
            popup_text = (f"<b>Primusschule:</b> {Primusschule['Primusschulname']}<br>"
                          f"<b>Adresse:</b> {', '.join(Primusschule['Adresse'])}<br>")
            popup = Popup(child=widgets.HTML(value=popup_text), location=(rechtswert, hochwert))
            icon = AwesomeIcon(name='school', marker_color='gray', icon_color='white')
            markerPrimusschule = Marker(location=(rechtswert, hochwert), draggable=False, popup=popup, icon=icon)
            Primusschule_markers.append(markerPrimusschule)
            m.add_layer(markerPrimusschule)
        except ValueError:
            print(f"Ungültige Koordinaten in Zeile: {Zeile}")
            pass
    else:
        print(f"Unvollständige Zeile: {Zeile}")

#-------------------------------------------------------------------------------------------------------------------------#
# Hauptschuldaten einlesen und organisieren
DataHauptschulen = []
with open("Hauptschule_DataFrame_Lena.csv", "r") as textFile:
    TextLines = textFile.readlines()
    TextLines = TextLines[1:]  # Entferne die erste Zeile
    for Zeile in TextLines:
        Zeile = Zeile.strip().split("\t")  # Entferne Leerzeichen und trenne Zeilen bei Tabs
        DataHauptschulen.append(Zeile)

Hauptschuldaten = {}
Hauptschule = {}
for Zeile in DataHauptschulen:
    if len(Zeile) > 8:  # Überprüfe, ob die Liste mindestens 9 Elemente enthält
        Hauptschule = {}
        Hauptschule["Hauptschulname"] = Zeile[1]
        Hauptschule["Adresse"] = Zeile[2:4]
        Hauptschule["Stadtteil"] = Zeile[5]
        Hauptschule["Rechtswert"] = Zeile[6]
        Hauptschule["Hochwert"] = Zeile[7]

        # Füge Marker an den Positionen der Hauptschulen in Münster hinzu
        try:
            rechtswert = float(Hauptschule["Rechtswert"])
            hochwert = float(Hauptschule["Hochwert"])
            adresse = ", ".join(zeile[2:4])  # Adresse in einen String umwandeln
            popup_text = (f"<b>Hauptschule:</b> {Hauptschule['Hauptschulname']}<br>"
                          f"<b>Adresse:</b> {', '.join(Hauptschule['Adresse'])}<br>")
            popup = Popup(child=widgets.HTML(value=popup_text), location=(rechtswert, hochwert))
            icon = AwesomeIcon(name='school', marker_color='beige', icon_color='white')
            markerHauptschule = Marker(location=(rechtswert, hochwert), draggable=False, popup=popup, icon=icon)
            Hauptschule_markers.append(markerHauptschule)
            m.add_layer(markerHauptschule)
        except ValueError:
            print(f"Ungültige Koordinaten in Zeile: {Zeile}")
            pass
    else:
        print(f"Unvollständige Zeile: {Zeile}")
#-------------------------------------------------------------------------------------------------------------------------#

# Realschuldaten einlesen und organisieren
DataRealschulen = []
with open("Realschulen_CSV.csv", "r") as textFile:
    TextLines = textFile.readlines()
    TextLines = TextLines[1:]  # Entferne die erste Zeile
    for i, Zeile in enumerate(TextLines):
        Zeile = Zeile.strip().split("\t")  # Entferne Leerzeichen und trenne Zeilen bei Tabulatoren
        DataRealschulen.append(Zeile)

Realschuldaten = {}
Realschule = {}
for i, Zeile in enumerate(DataRealschulen):
    if len(Zeile) >= 5:  # Überprüfe, ob die Liste mindestens 7 Elemente enthält
        Realschule = {}
        Realschule["Realschulname"] = Zeile[0]
        Realschule["Adresse"] = Zeile[2]
        Realschule["Stadtteil"] = Zeile[3]
        Realschule["Rechtswert"] = Zeile[4]
        Realschule["Hochwert"] = Zeile[5]

        # Füge Marker an den Positionen der Realschulen in Münster hinzu
        try:
            rechtswert = float(Realschule["Rechtswert"])
            hochwert = float(Realschule["Hochwert"])
            popup_text = (f"<b>Realschule:</b> {Realschule['Realschulname']}<br>"
                          f"<b>Adresse:</b> {Realschule['Adresse']}<br>")
            popup = Popup(child=widgets.HTML(value=popup_text), location=(rechtswert, hochwert))
            icon = AwesomeIcon(name='school', marker_color='lightblue', icon_color='white')
            markerRealschule = Marker(location=(rechtswert, hochwert), draggable=False, popup=popup, icon=icon)
            Realschule_markers.append(markerRealschule)
            m.add_layer(markerRealschule)
        except ValueError:
            print(f"Ungültige Koordinaten in Zeile")
            pass
    else:
        print(f"Unvollständige Zeile")
#-------------------------------------------------------------------------------------------------------------------------#

# Gesamtschuldaten einlesen und organisieren
DataGesamtschulen = []
with open("Gesamtschulen_CSV.csv", "r") as textFile:
    TextLines = textFile.readlines()
    TextLines = TextLines[1:]  # Entferne die erste Zeile
    for i, Zeile in enumerate(TextLines):
        Zeile = Zeile.strip().split("\t")  # Entferne Leerzeichen und trenne Zeilen bei Tabulatoren
        DataGesamtschulen.append(Zeile)

Gesamtschuldaten = {}
Gesamtschule = {}
for i, Zeile in enumerate(DataGesamtschulen):
    if len(Zeile) >= 5:  # Überprüfe, ob die Liste mindestens 7 Elemente enthält
        Gesamtschule = {}
        Gesamtschule["Gesamtschulname"] = Zeile[0]
        Gesamtschule["Adresse"] = Zeile[2]
        Gesamtschule["Stadtteil"] = Zeile[3]
        Gesamtschule["Rechtswert"] = Zeile[4]
        Gesamtschule["Hochwert"] = Zeile[5]

        # Füge Marker an den Positionen der Gesamtschulen in Münster hinzu
        try:
            rechtswert = float(Gesamtschule["Rechtswert"])
            hochwert = float(Gesamtschule["Hochwert"])
            popup_text = (f"<b>Gesamtschule:</b> {Gesamtschule['Gesamtschulname']}<br>"
                          f"<b>Adresse:</b> {Gesamtschule['Adresse']}<br>")
            popup = Popup(child=widgets.HTML(value=popup_text), location=(rechtswert, hochwert))
            icon = AwesomeIcon(name='school', marker_color='red', icon_color='white')
            markerGesamtschule = Marker(location=(rechtswert, hochwert), draggable=False, popup=popup, icon=icon)
            Gesamtschule_markers.append(markerGesamtschule)
            m.add_layer(markerGesamtschule)
        except ValueError:
            print(f"Ungültige Koordinaten in Zeile")
            pass
    else:
        print(f"Unvollständige Zeile")
#-------------------------------------------------------------------------------------------------------------------------#

# Gymnasiendaten einlesen und organisieren
DataGymnasium = []
with open("Gymnasium_CSV.csv", "r") as textFile:
    TextLines = textFile.readlines()
    TextLines = TextLines[1:]  # Entferne die erste Zeile
    for i, Zeile in enumerate(TextLines):
        Zeile = Zeile.strip().split("\t")  # Entferne Leerzeichen und trenne Zeilen bei Tabulatoren
        DataGymnasium.append(Zeile)

Gymnasiumdaten = {}
Gymnasium = {}
Gymnasium_markers = []  # Stelle sicher, dass du die Liste initialisierst
for i, Zeile in enumerate(DataGymnasium):
    if len(Zeile) >= 5:  # Überprüfe, ob die Liste mindestens 7 Elemente enthält
        Gymnasium = {}
        Gymnasium["Gymnasiumname"] = Zeile[0]
        Gymnasium["Adresse"] = Zeile[2]
        Gymnasium["Stadtteil"] = Zeile[3]
        Gymnasium["Rechtswert"] = Zeile[4]
        Gymnasium["Hochwert"] = Zeile[5]

        # Füge Marker an den Positionen der Gymnasien in Münster hinzu
        try:
            rechtswert = float(Gymnasium["Rechtswert"])
            hochwert = float(Gymnasium["Hochwert"])
            popup_text = (f"<b>Gymnasium:</b> {Gymnasium['Gymnasiumname']}<br>"
                          f"<b>Adresse:</b> {Gymnasium['Adresse']}<br>")
            popup = Popup(child=widgets.HTML(value=popup_text), location=(rechtswert, hochwert))
            icon = AwesomeIcon(name='school', marker_color='darkgreen', icon_color='white')
            markerGymnasium = Marker(location=(rechtswert, hochwert), draggable=False, popup=popup, icon=icon)
            Gymnasium_markers.append(markerGymnasium)
            m.add_layer(markerGymnasium)
        except ValueError:
            print(f"Ungültige Koordinaten in Zeile")
            pass
    else:
        print(f"Unvollständige Zeile")

# Checkboxen für die Legende ohne HTML
kita_checkbox = Checkbox(value=True, indent=False, layout=Layout(width='20px'))
bibliothek_checkbox = Checkbox(value=True, indent=False, layout=Layout(width='20px'))
buecherbus_checkbox = Checkbox(value=True, indent=False, layout=Layout(width='20px'))
Grundschule_checkbox = Checkbox(value=True, indent=False, layout=Layout(width='20px'))
Primusschule_checkbox = Checkbox(value=True, indent=False, layout=Layout(width='20px'))
Förderschule_checkbox = Checkbox(value=True, indent=False, layout=Layout(width='20px'))
Hauptschule_checkbox = Checkbox(value=True, indent=False, layout=Layout(width='20px'))
Realschule_checkbox = Checkbox(value=True, indent=False, layout=Layout(width='20px'))
Gesamtschule_checkbox = Checkbox(value=True, indent=False, layout=Layout(width='20px'))
Gymnasium_checkbox = Checkbox(value=True, indent=False, layout=Layout(width='20px'))

# Funktion zum Ein- und Ausblenden der Marker basierend auf den Checkboxen
def update_markers(change):
    for marker in kita_markers:
        marker.visible = kita_checkbox.value
        if kita_checkbox.value:
            marker.popup = Popup(child=widgets.HTML(value=marker.popup.child.value), location=marker.location)
        else:
            marker.popup = None
    for marker in bibliothek_markers:
        marker.visible = bibliothek_checkbox.value
        if bibliothek_checkbox.value:
            marker.popup = Popup(child=widgets.HTML(value=marker.popup.child.value), location=marker.location)
        else:
            marker.popup = None
    for marker in buecherbus_markers:
        marker.visible = buecherbus_checkbox.value
        if buecherbus_checkbox.value:
            marker.popup = Popup(child=widgets.HTML(value=marker.popup.child.value), location=marker.location)
        else:
            marker.popup = None
    for marker in Grundschule_markers:
        marker.visible = Grundschule_checkbox.value
        if Grundschule_checkbox.value:
            marker.popup = Popup(child=widgets.HTML(value=marker.popup.child.value), location=marker.location)
        else:
            marker.popup = None
    for marker in Förderschule_markers:
        marker.visible = Förderschule_checkbox.value
        if Förderschule_checkbox.value:
            marker.popup = Popup(child=widgets.HTML(value=marker.popup.child.value), location=marker.location)
        else:
            marker.popup = None
    for marker in Primusschule_markers:
        marker.visible = Primusschule_checkbox.value
        if Primusschule_checkbox.value:
            marker.popup = Popup(child=widgets.HTML(value=marker.popup.child.value), location=marker.location)
        else:
            marker.popup = None
    for marker in Hauptschule_markers:
        marker.visible = Hauptschule_checkbox.value
        if Hauptschule_checkbox.value:
            marker.popup = Popup(child=widgets.HTML(value=marker.popup.child.value), location=marker.location)
        else:
            marker.popup = None
    for marker in Realschule_markers:
        marker.visible = Realschule_checkbox.value
        if Realschule_checkbox.value:
            marker.popup = Popup(child=widgets.HTML(value=marker.popup.child.value), location=marker.location)
        else:
            marker.popup = None
    for marker in Gesamtschule_markers:
        marker.visible = Gesamtschule_checkbox.value
        if Gesamtschule_checkbox.value:
            marker.popup = Popup(child=widgets.HTML(value=marker.popup.child.value), location=marker.location)
        else:
            marker.popup = None
    for marker in Gymnasium_markers:
        marker.visible = Gymnasium_checkbox.value
        if Gymnasium_checkbox.value:
            marker.popup = Popup(child=widgets.HTML(value=marker.popup.child.value), location=marker.location)
        else:
            marker.popup = None

kita_checkbox.observe(update_markers, names='value')
bibliothek_checkbox.observe(update_markers, names='value')
buecherbus_checkbox.observe(update_markers, names='value')
Grundschule_checkbox.observe(update_markers, names='value')
Primusschule_checkbox.observe(update_markers, names='value')
Förderschule_checkbox.observe(update_markers, names='value')
Hauptschule_checkbox.observe(update_markers, names='value')
Realschule_checkbox.observe(update_markers, names='value')
Gesamtschule_checkbox.observe(update_markers, names='value')
Gymnasium_checkbox.observe(update_markers, names='value')

# Erstelle die Legende mit Checkboxen und HTML
legend_html = """
<div style="background-color: white; border: 1px solid grey; padding: 10px;">
    <h4>Legende</h4>
    <div id="legend-checkboxes"></div>
</div>
"""
legend_widget = HTML(value=legend_html)
legend_control = WidgetControl(widget=legend_widget, position='topright')

# Füge das WidgetControl der Karte hinzu
m.add_control(legend_control)

# Füge die Checkboxen der Legende hinzu
legend_vbox = VBox([
    HBox([kita_checkbox, HTML(value='<div style="padding-left: 5px;"><i class="fa fa-rainbow" style="color: purple;"></i> Kita</div>')]),
    HBox([bibliothek_checkbox, HTML(value='<div style="padding-left: 5px;"><i class="fa fa-book" style="color: darkblue;"></i> Bücherei</div>')]),
    HBox([buecherbus_checkbox, HTML(value='<div style="padding-left: 5px;"><i class="fa fa-bus" style="color: green;"></i> Bücherbus</div>')]),
    HBox([Grundschule_checkbox, HTML(value='<div style="padding-left: 5px;"><i class="fa fa-school" style="color: black;"></i> Grundschule</div>')]),
    HBox([Primusschule_checkbox, HTML(value='<div style="padding-left: 5px;"><i class="fa fa-school" style="color: gray;"></i> Primusschule</div>')]),
    HBox([Förderschule_checkbox, HTML(value='<div style="padding-left: 5px;"><i class="fa fa-school" style="color: orange;"></i> Förderschule</div>')]),
    HBox([Hauptschule_checkbox, HTML(value='<div style="padding-left: 5px;"><i class="fa fa-school" style="color: beige;"></i> Hauptschule</div>')]),
    HBox([Realschule_checkbox, HTML(value='<div style="padding-left: 5px;"><i class="fa fa-school" style="color: lightblue;"></i> Realschule</div>')]),
    HBox([Gesamtschule_checkbox, HTML(value='<div style="padding-left: 5px;"><i class="fa fa-school" style="color: red;"></i> Gesamtschule</div>')]),
    HBox([Gymnasium_checkbox, HTML(value='<div style="padding-left: 5px;"><i class="fa fa-school" style="color: darkgreen;"></i> Gymnasium</div>')])
], layout=Layout(margin='0 0 0 10px'))


legend_control.widget = VBox([legend_vbox])

# Zeige die Karte


IndexError: list index out of range

In [2]:
# Speichere die Karte als HTML-Datei
from ipyleaflet import Map
m.save("Karte.html")

